the code in this notebook comes from:
https://gist.github.com/DollarAkshay/55e50158a1259cb4ef5675a427bf1781

In [4]:
import time, math, random, bisect
import gym
import numpy as np
import csv

In [13]:
MAX_GENERATIONS = 100
MAX_STEPS = 500 
POPULATION_COUNT = 200
MUTATION_RATE = 0.001
PLAY = 100

In [14]:
def sigmoid(x):
    return 1.0/(1.0 + np.exp(-x))

class NeuralNet :
    
    def __init__(self, nodeCount):     
        self.fitness = 0
        self.nodeCount = nodeCount
        self.weights = []
        self.biases = []
        for i in range(len(nodeCount) - 1):
            self.weights.append( np.random.uniform(low=-1, high=1, size=(nodeCount[i], nodeCount[i+1])).tolist() )
            self.biases.append( np.random.uniform(low=-1, high=1, size=(nodeCount[i+1])).tolist())


    def printWeightsandBiases(self):
        
        print("--------------------------------")
        print("Weights :\n[", end="")
        for i in range(len(self.weights)):
            print("\n [ ", end="")
            for j in range(len(self.weights[i])):
                if j!=0:
                    print("\n   ", end="")
                print("[", end="")
                for k in range(len(self.weights[i][j])):
                    print(" %5.2f," % (self.weights[i][j][k]), end="")
                print("\b],", end="")
            print("\b ],")
        print("\n]")

        print("\nBiases :\n[", end="")
        for i in range(len(self.biases)):
            print("\n [ ", end="")
            for j in range(len(self.biases[i])):
                    print(" %5.2f," % (self.biases[i][j]), end="")
            print("\b],", end="")
        print("\b \n]\n--------------------------------\n")


  
    def getOutput(self, input):
        output = input
        for i in range(len(self.nodeCount)-1):
            output = np.reshape( np.matmul(output, self.weights[i]) + self.biases[i], (self.nodeCount[i+1]))
        return np.argmax(sigmoid(output))

In [15]:
class Population :

    def __init__(self, populationCount, mutationRate, nodeCount):
        self.nodeCount = nodeCount
        self.popCount = populationCount
        self.m_rate = mutationRate
        self.population = [ NeuralNet(nodeCount) for i in range(populationCount)]


    def createChild(self, nn1, nn2):
        
        child = NeuralNet(self.nodeCount)

        for i in range(len(child.weights)):
            for j in range(len(child.weights[i])):
                for k in range(len(child.weights[i][j])):
                    if random.random() < self.m_rate:
                        child.weights[i][j][k] = random.uniform(-1, 1)
                    else:
                        child.weights[i][j][k] = (nn1.weights[i][j][k] + nn2.weights[i][j][k])/2.0

        for i in range(len(child.biases)):
            for j in range(len(child.biases[i])):
                if random.random() < self.m_rate:
                    child.biases[i][j] = random.uniform(-1, 1)
                else:
                    child.biases[i][j] = (nn1.biases[i][j] + nn2.biases[i][j])/2.0

        return child


    def createNewGeneration(self):       
        nextGen = []
        fitnessSum = [0]
        for i in range(len(self.population)):
            fitnessSum.append(fitnessSum[i]+self.population[i].fitness)
        
        while(len(nextGen) < self.popCount):
            r1 = random.uniform(0, fitnessSum[len(fitnessSum)-1] )
            r2 = random.uniform(0, fitnessSum[len(fitnessSum)-1] )
            nn1 = self.population[bisect.bisect_right(fitnessSum, r1)-1]
            nn2 = self.population[bisect.bisect_right(fitnessSum, r2)-1]
            nextGen.append( self.createChild(nn1, nn2) )
        self.population.clear()
        self.population = nextGen       

In [16]:
env = gym.make('CartPole-v1')
observation = env.reset()

in_dimen = env.observation_space.shape[0]
out_dimen = env.action_space.n
pop = Population(POPULATION_COUNT, MUTATION_RATE, [in_dimen, 8, 8, out_dimen])

bestNeuralNets = []

with open("Cartpole_GA_train_scores.csv", "w") as csvfile:
    writer = csv.writer(csvfile, delimiter=',')
    header = ["generation", "average fitness", "max fitness"]
    writer.writerow(header)
    for gen in range(MAX_GENERATIONS):
        genAvgFit = 0.0
        maxFit = 0.0
        maxNeuralNet = None
        for nn in pop.population:
            totalReward = 0
            for step in range(MAX_STEPS):
                action = nn.getOutput(observation)
                observation, reward, done, info = env.step(action)
                totalReward += reward
                if done:
                    observation = env.reset()
                    break
            nn.fitness = totalReward
            genAvgFit += nn.fitness
            if nn.fitness > maxFit :
                maxFit = nn.fitness
                maxNeuralNet = nn

        bestNeuralNets.append(maxNeuralNet)
        genAvgFit/=pop.popCount
        print("Generation : %3d |  Avg Fitness : %4.0f  |  Max Fitness : %4.0f  " % (gen+1, genAvgFit, maxFit) )
        writer.writerow([gen+1, genAvgFit, maxFit])

        pop.createNewGeneration()


fitnesses = []
for i in range(len(bestNeuralNets)):
    fitnesses.append(bestNeuralNets[i].fitness)  

found = False

for i in range(len(bestNeuralNets)):
    if not found:
        if bestNeuralNets[i].fitness == max(fitnesses):
            best_nn = bestNeuralNets[i]
            found = True

            
with open("Cartpole_GA_play_scores.csv", "w") as csvfile2:
    writer = csv.writer(csvfile2, delimiter=',')
    header = ["attempt", "score"]
    writer.writerow(header)
    print("-------------------------------------------------------------")
    for attempt in range(PLAY):
        rewards = 0
        for step in range(MAX_STEPS):
            action = best_nn.getOutput(observation)
            observation, reward, done, info = env.step(action)
            rewards += reward
            if done:
                observation = env.reset()
                break
       
        print("Attempt : %3d |  Score : %4.0f  " % (attempt+1, rewards) )

        writer.writerow([attempt+1, rewards])
        

Generation :   1 |  Avg Fitness :   18  |  Max Fitness :  500  
Generation :   2 |  Avg Fitness :   44  |  Max Fitness :  500  
Generation :   3 |  Avg Fitness :  167  |  Max Fitness :  500  
Generation :   4 |  Avg Fitness :  372  |  Max Fitness :  500  
Generation :   5 |  Avg Fitness :  465  |  Max Fitness :  500  
Generation :   6 |  Avg Fitness :  488  |  Max Fitness :  500  
Generation :   7 |  Avg Fitness :  480  |  Max Fitness :  500  
Generation :   8 |  Avg Fitness :  483  |  Max Fitness :  500  
Generation :   9 |  Avg Fitness :  491  |  Max Fitness :  500  
Generation :  10 |  Avg Fitness :  484  |  Max Fitness :  500  
Generation :  11 |  Avg Fitness :  489  |  Max Fitness :  500  
Generation :  12 |  Avg Fitness :  496  |  Max Fitness :  500  
Generation :  13 |  Avg Fitness :  479  |  Max Fitness :  500  
Generation :  14 |  Avg Fitness :  487  |  Max Fitness :  500  
Generation :  15 |  Avg Fitness :  482  |  Max Fitness :  500  
Generation :  16 |  Avg Fitness :  487  

Attempt :  59 |  Score :  500  
Attempt :  60 |  Score :  500  
Attempt :  61 |  Score :  500  
Attempt :  62 |  Score :  500  
Attempt :  63 |  Score :  500  
Attempt :  64 |  Score :  500  
Attempt :  65 |  Score :  500  
Attempt :  66 |  Score :  500  
Attempt :  67 |  Score :  500  
Attempt :  68 |  Score :  500  
Attempt :  69 |  Score :  500  
Attempt :  70 |  Score :  500  
Attempt :  71 |  Score :  500  
Attempt :  72 |  Score :  500  
Attempt :  73 |  Score :  500  
Attempt :  74 |  Score :  500  
Attempt :  75 |  Score :  500  
Attempt :  76 |  Score :  500  
Attempt :  77 |  Score :  500  
Attempt :  78 |  Score :  500  
Attempt :  79 |  Score :  500  
Attempt :  80 |  Score :  500  
Attempt :  81 |  Score :  500  
Attempt :  82 |  Score :  500  
Attempt :  83 |  Score :  500  
Attempt :  84 |  Score :  500  
Attempt :  85 |  Score :  500  
Attempt :  86 |  Score :  500  
Attempt :  87 |  Score :  500  
Attempt :  88 |  Score :  500  
Attempt :  89 |  Score :  500  
Attempt 